In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,mar-19,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,mar-19,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,mar-19,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,mar-19,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,mar-19,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
dfx = pd.get_dummies(df,columns=["home_ownership","verification_status","issue_d","initial_list_status","next_pymnt_d","application_type","loan_status","pymnt_plan","hardship_flag","debt_settlement_flag"])

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2 = df.copy()
df2['loan_status'] = le.fit_transform(df2['loan_status']) 
df2.tail(60)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68757,30000.0,0.1072,978.19,MORTGAGE,120000.0,Not Verified,Jan-2019,1,n,18.83,...,88.5,60.0,0.0,0.0,372691.0,36955.0,17200.0,37985.0,N,N
68758,12000.0,0.2340,467.03,RENT,60000.0,Not Verified,Jan-2019,1,n,21.65,...,91.3,40.0,1.0,0.0,58857.0,46795.0,9200.0,37857.0,N,N
68759,17000.0,0.1614,598.85,MORTGAGE,77500.0,Source Verified,Jan-2019,1,n,27.92,...,100.0,66.7,0.0,0.0,509343.0,388437.0,21300.0,298143.0,N,N
68760,12000.0,0.1992,445.48,MORTGAGE,184992.0,Source Verified,Jan-2019,1,n,15.90,...,88.0,0.0,0.0,0.0,460191.0,67159.0,11500.0,78631.0,N,N
68761,25000.0,0.1992,928.08,RENT,68000.0,Source Verified,Jan-2019,1,n,15.16,...,100.0,0.0,0.0,0.0,31200.0,17974.0,10500.0,20500.0,N,N
68762,16000.0,0.1447,550.51,MORTGAGE,110000.0,Source Verified,Jan-2019,1,n,33.54,...,97.0,33.3,0.0,0.0,354485.0,169610.0,24600.0,160762.0,N,N
68763,4500.0,0.1356,152.84,MORTGAGE,66000.0,Verified,Jan-2019,1,n,15.47,...,62.5,33.3,0.0,0.0,198814.0,21219.0,7900.0,22950.0,N,N
68764,33500.0,0.1502,797.32,MORTGAGE,77276.0,Source Verified,Jan-2019,1,n,15.98,...,100.0,0.0,0.0,0.0,390950.0,54212.0,19400.0,46000.0,N,N
68765,15200.0,0.1797,549.29,MORTGAGE,59000.0,Not Verified,Jan-2019,1,n,10.78,...,90.0,100.0,1.0,0.0,86949.0,20205.0,9000.0,8849.0,N,N
68766,24000.0,0.1894,621.79,RENT,130000.0,Not Verified,Jan-2019,1,n,13.91,...,86.7,66.7,1.0,0.0,87768.0,74472.0,16400.0,67319.0,N,N


In [8]:
df2 = pd.get_dummies(df,columns=["home_ownership","verification_status","issue_d","initial_list_status","next_pymnt_d","application_type","pymnt_plan","hardship_flag","debt_settlement_flag"])
# Create our features
X = df2.drop('loan_status',axis=1)

# Create our target
y = df2

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_mar-19,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_may-19,application_type_Individual,application_type_Joint App,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0


In [10]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100, n_classes=3,
                           n_informative=4, weights=[0.2, 0.3, 0.5],
                           random_state=0)
clf = BalancedRandomForestClassifier(random_state=1)
clf.fit(X, y)
importances = clf.feature_importances_
print(importances)
predictions = clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                    0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print(predictions)

[0.03497095 0.03026441 0.03651004 0.0360675  0.11090666 0.03003357
 0.03055822 0.02956232 0.03181858 0.03729848 0.15186307 0.02719581
 0.03334068 0.04227633 0.05104677 0.03725789 0.05440022 0.03021146
 0.12577677 0.03864028]
[1]


In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score

acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

ValueError: Found input variables with inconsistent numbers of samples: [17205, 1]

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

In [ ]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report

print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# List the features sorted in descending order by feature importance
cm

### Easy Ensemble AdaBoost Classifier

In [14]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

X, y = make_classification(random_state=1)
eec = EasyEnsembleClassifier(random_state=1)

# Fit the model
eec.fit(X_train_scaled, y_train)

# Make Prediction
y_pred  = ecc.predict(X_test_scaled)
pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).head()

TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score

acc_score = accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {acc_score}")

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

In [ ]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report

print("Classification Report")
print(classification_report(y_test, y_pred))